# Explanability-Driven In-context Learning

# Importing the required modules

In [1]:
# modules used for data handling
import pandas as pd
import numpy as np
import json
import random

# modules used for modeling
from xgboost import XGBClassifier

# user-defined modules
from scripts.configs import Dataset, Model
from scripts.preprocess import preprocess_titanic
from scripts.prompt_generator import (zero_shot_prompt_generator, 
                                      reasoning_generator_prompt,
                                      objective_judge_prompt_generator)

from scripts.explanable_tree_model import ExplainableModel
from scripts.zero_shot_baseline import ZeroShotBaseline
from scripts.diverse_examples import get_diverse_examples
from scripts.reason_generation import ReasonGenerator
from scripts.objective_judge import ObjectiveJudge

# modules used for env variables
import os
from dotenv import load_dotenv
load_dotenv()

WANDB_API_KEY = os.getenv("WANDB_API_KEY")
WANDB_PROJECT_NAME = os.getenv("WANDB_PROJECT_NAME")
PROJECT_NAME = os.getenv("PROJECT_NAME")
BUCKET_NAME = os.getenv("BUCKET_NAME")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assign

In [2]:
titanic_dataset = Dataset(
    name="titanic",
    path="data/datasets/titanic.csv",
    config_file_path="data/dataset_config/titanic_config.json",
    preprocess_fn=preprocess_titanic,
    shap_vals_path="data/shap_values/titanic_shap.csv",
    target_col="Survived"
)

# Tree Model Performance (XGBoost)

In [3]:
clf = XGBClassifier()
xmodel = ExplainableModel(
    dataset=titanic_dataset,
    estimator=clf
)
xmodel.explain(params_grid_file="data/tune_config/xgb.json")

[Titanic] Dropped 179 rows due to NaNs (kept 712 rows).
Create sweep with ID: yc32vi2s
Sweep URL: https://wandb.ai/gauravpendharkar/xai-guided-cot/sweeps/yc32vi2s


wandb: Agent Starting Run: 1jdbwahq with config:
wandb: 	learning_rate: 0.27854961660395194
wandb: 	max_depth: 6
wandb: 	min_child_weight: 1
wandb: 	n_estimators: 339
wandb: 	random_state: 42
wandb: 	reg_lambda: 0.08415814901838715
wandb: 	subsample: 0.6877590380220275
wandb: Currently logged in as: mitugaurav15 (gauravpendharkar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [anthropic, google.genai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


accuracy,▁
accuracy_std,▁
f1_macro,▁
f1_macro_std,▁
accuracy,0.79801
accuracy_std,0.03593
f1_macro,0.7885
f1_macro_std,0.03672


wandb: Agent Starting Run: 9wwfoqxj with config:
wandb: 	learning_rate: 0.1139874026806906
wandb: 	max_depth: 3
wandb: 	min_child_weight: 3
wandb: 	n_estimators: 400
wandb: 	random_state: 42
wandb: 	reg_lambda: 0.9535245332694242
wandb: 	subsample: 0.7282078377467783


...

wandb: Agent Starting Run: w4juistf with config:
wandb: 	learning_rate: 0.06860036760191021
wandb: 	max_depth: 3
wandb: 	min_child_weight: 5
wandb: 	n_estimators: 363
wandb: 	random_state: 42
wandb: 	reg_lambda: 1.8351968143525192
wandb: 	subsample: 0.72028470097465


accuracy,▁
accuracy_std,▁
f1_macro,▁
f1_macro_std,▁
accuracy,0.81379
accuracy_std,0.04102
f1_macro,0.80039
f1_macro_std,0.04393


Completed hyperparameter tuning.


wandb: Sorting runs by -summary_metrics.f1_macro


Trained model with best hyperparameters.
Logged explanation data to data/dataset_config/titanic_config.json
Explanation process completed.


# Zero Shot Baseline

In [ ]:
dataset = Dataset(name="titanic", path="data/datasets/titanic.csv", target_column="Survived")
model = Model(name="gemini-2.5-flash", temperature=0.0, max_tokens=150)

baseline = ZeroShotBaseline(dataset=dataset, 
                            model=model, 
                            prompt_gen_fn=zero_shot_prompt_generator)


# Reason Generation

In [3]:
reasoning_model = Model(
    name="deepseek-ai/DeepSeek-R1",
    temperature=0.6,
    max_tokens=4096
)

rg = ReasonGenerator(
    dataset=titanic_dataset,
    model=reasoning_model,
    prompt_gen_fn=reasoning_generator_prompt
)

rg.create_batch_prompts()

Found best number of clusters: k=3 with silhouette score: 0.2747174239109675
Chosen 3 diverse examples.
[Titanic] Dropped 179 rows due to NaNs (kept 712 rows).


In [5]:
rg.save_batches_as_jsonl()

In [6]:
rg.submit_batches()

Uploading file titanic_reasoning_batches.jsonl: 100%|██████████| 6.40k/6.40k [00:00<00:00, 10.2kB/s]


Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.IN_PROGRESS
Current Status: BatchJobStatus.COMPLETED
Batch completed successfully.


In [ ]:
results_jsonl_path = f"data/batch_outputs/{titanic_dataset.name}_reasoning_predictions.jsonl"
results = []
with open(results_jsonl_path, 'r') as f:
    for line in f:
        results.append(json.loads(line.strip()))

In [20]:
from scripts.postprocess import parse_reasoning_llm_results
parse_reasoning_llm_results(results_jsonl_path)

{862: "The model correctly predicts survival (1) for this passenger, matching the ground truth label (1.0). The prediction is primarily driven by two dominant features: **Sex** (SHAP +2.37) and **Pclass** (SHAP +1.97). The high positive SHAP value for Sex=1.0 (likely indicating female, as females had higher survival rates) strongly increases survival probability, consistent with the dataset's top feature importance (0.44). Similarly, Pclass=1.0 (first class) contributes significantly to survival due to prioritized evacuation. Supporting features like **Fare** (SHAP +0.21, moderate cost aligning with first-class status) and **SibSp=0.0** (SHAP +0.10, no siblings/spouses competing for resources) provide additional positive contributions. Although **Age=48.0** (SHAP +0.05) slightly favors survival (possibly due to adulthood and evacuation priority), and **Parch=0.0** (SHAP -0.02, no children/parents) has a negligible negative effect, their impacts are dwarfed by the advantages of Sex and 

# LLM as Judge

# CoT-based Classification